## Imports

In [25]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
import csv
from time import sleep
from tqdm import tqdm



client_id = 'ca8e1b5257a34872a23454ea7cfd85c8'
client_secret = '93436bda55ce4970b1c9abbf5be56d70'

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)


## Get us_categories from json file

In [2]:
import json
from pprint import pprint as pp

# Specify the file path
file_path = 'us_categories.json'

# Open the file and load the JSON data
with open(file_path, 'r') as file:
    us_categories = json.load(file)

# Print the loaded data
pp(us_categories)


{'0JQ5DAqbMKFA6SOHvT3gck': 'Party',
 '0JQ5DAqbMKFAJ5xb0fwo9m': 'Jazz',
 '0JQ5DAqbMKFAUsdyVjCQuL': 'Romance',
 '0JQ5DAqbMKFAXlCG6QvYQ4': 'Workout',
 '0JQ5DAqbMKFAjfauKLOZiv': 'Punk',
 '0JQ5DAqbMKFCWjUTdzaG0e': 'Indie',
 '0JQ5DAqbMKFCbimwdOYlsl': 'Focus',
 '0JQ5DAqbMKFCfObibaOZbv': 'Gaming',
 '0JQ5DAqbMKFCuoRTxhYWow': 'Sleep',
 '0JQ5DAqbMKFDBgllo2cUIN': 'Spotify Singles',
 '0JQ5DAqbMKFDTEtSaS4R92': 'Regional Mexican',
 '0JQ5DAqbMKFDXXwE9BDJAr': 'Rock',
 '0JQ5DAqbMKFDkd668ypn6O': 'Metal',
 '0JQ5DAqbMKFEC4WFtoNRpw': 'Pop',
 '0JQ5DAqbMKFEOEBCABAxo9': 'Netflix',
 '0JQ5DAqbMKFEZPnFQSFB1T': 'R&B',
 '0JQ5DAqbMKFF9bY76LXmfI': 'Frequency',
 '0JQ5DAqbMKFFoimhOqWzLB': 'Kids & Family',
 '0JQ5DAqbMKFFsW9N8maB6z': 'Funk & Disco',
 '0JQ5DAqbMKFFtlLYUHv8bT': 'Alternative',
 '0JQ5DAqbMKFFzDl7qN9Apr': 'Chill',
 '0JQ5DAqbMKFGvOw3O4nLAf': 'K-Pop',
 '0JQ5DAqbMKFHOzuVTgTizF': 'Dance/Electronic',
 '0JQ5DAqbMKFIRybaNTYXXy': 'In the car',
 '0JQ5DAqbMKFIVNxQgRNSg0': 'Decades',
 '0JQ5DAqbMKFImHYGo3eTSg': 'Fresh Fi

## Get set of Playlist IDs

In [3]:
import pandas as pd

Playlists = pd.read_csv('playlists_per_category.csv', index_col=0)

Playlists.head()

unique_playlists = Playlists.stack().unique()
print(len(unique_playlists))


2886


## Create a dictionary of every song id in each playlist

In [26]:
# Get all tracks from each playlist in each category
playlists_songs = {}
for playlist_id in tqdm(unique_playlists):
    offset = 0
    print(playlist_id)
    if playlist_id in playlists_songs.keys():
        continue
    playlists_songs[playlist_id] = []
    while True:
        # sleep(1)
        # print(playlist_id)
        try:
            print(f"Break incoming for: {playlist_id} and {offset}\n")
            playlist = sp.playlist_tracks(playlist_id=playlist_id,
                                         limit=100,
                                         offset=offset)
            print('api call happened')
            tracks = playlist['items']
            for track in tracks:
                if type(track['track']) is type(None):
                    break
                playlists_songs[playlist_id].append(track['track']['id'])
            if playlist['next'] is not None:
                offset += 100
            else:
                break
        except spotipy.SpotifyException as e:
            print(e)
            if e.http_status == 401 and 'access token expired' in e.msg.lower():
                print('Refreshing token')
                sp.client_credentials_manager.refresh_token(auth_manager.client_id,
                                                            auth_manager.client_secret)
            if e.http_status == 429:
                print('Sleeping for 10 seconds')
                sleep(10)
        except Exception as e:
            print(e)
            print(playlist_id)
            break
pprint(playlists_songs)


  0%|          | 0/2886 [00:00<?, ?it/s]

37i9dQZF1DXcBWIGoYBM5M
Break incoming for: 37i9dQZF1DXcBWIGoYBM5M and 0



  0%|          | 0/2886 [00:08<?, ?it/s]


KeyboardInterrupt: 

In [72]:
# pprint(playlists_songs)
print(playlists_songs.keys())


dict_keys(['37i9dQZF1DXcBWIGoYBM5M', '37i9dQZF1DX0XUsuxWHRQd'])


In [65]:
# df = pd.DataFrame.from_dict(playlists_songs, orient='index')
# df
# playlists_songs = df.to_dict(orient='index')
# pprint(playlists_songs)
copy = playlists_songs.copy()


In [92]:
import requests

birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
response = sp.artist_albums(birdy_uri, album_type='album')
print(response)
print(response.status_code)
print(response.headers.get('X-RateLimit-Limit'))
print(response.headers.get('X-RateLimit-Remaining'))
print(response.headers.get('X-RateLimit-Reset'))


{'href': 'https://api.spotify.com/v1/artists/2WX2uTcsvV5OnS0inACecP/albums?include_groups=album&offset=0&limit=20', 'items': [{'album_group': 'album', 'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2WX2uTcsvV5OnS0inACecP'}, 'href': 'https://api.spotify.com/v1/artists/2WX2uTcsvV5OnS0inACecP', 'id': '2WX2uTcsvV5OnS0inACecP', 'name': 'Birdy', 'type': 'artist', 'uri': 'spotify:artist:2WX2uTcsvV5OnS0inACecP'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', '

AttributeError: 'dict' object has no attribute 'status_code'

In [18]:
sp.search(q='playlist:37i9dQZF1DXcBWIGoYBM5M', type='playlist')


{'playlists': {'href': 'https://api.spotify.com/v1/search?query=playlist%3A37i9dQZF1DXcBWIGoYBM5M&type=playlist&offset=0&limit=10',
  'items': [{'collaborative': False,
    'description': 'spotify:user:spotify:playlist:37i9dQZF1DXcBWIGoYBM5M',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/612zawZmFOIKjWuIYUro3V'},
    'href': 'https://api.spotify.com/v1/playlists/612zawZmFOIKjWuIYUro3V',
    'id': '612zawZmFOIKjWuIYUro3V',
    'images': [{'height': 640,
      'url': 'https://mosaic.scdn.co/640/ab67616d0000b27313584782bb6d165483d296edab67616d0000b273352b3ced3f510b351e02b058ab67616d0000b2735f76dbd311a0782823550b30ab67616d0000b273bf01fd0986a195d485922167',
      'width': 640},
     {'height': 300,
      'url': 'https://mosaic.scdn.co/300/ab67616d0000b27313584782bb6d165483d296edab67616d0000b273352b3ced3f510b351e02b058ab67616d0000b2735f76dbd311a0782823550b30ab67616d0000b273bf01fd0986a195d485922167',
      'width': 300},
     {'height': 60,
      'url': 'https://mosaic.